In [75]:
from catboost import CatBoostClassifier, CatBoostRegressor
import sys
sys.path.insert(0, '../utils')
from utils.football_data_set_splitter import FootballDataSetSplitter
import utils.football_data_set as fds
import pandas as pd

In [3]:
train = fds.FootballDataSet("../data/train.csv")

In [4]:
test = fds.FootballDataSet("../data/test.csv")

In [6]:
categorical_features = train.get_categorical_columns()
categorical_features

['winner',
 'team',
 'player_general_var_0',
 'player_general_var_1',
 'player_general_var_5',
 'player_positional_raw_var_1',
 'player_positional_raw_var_2',
 'player_positional_raw_var_3',
 'player_defensive_raw_var_5',
 'player_offensive_raw_var_7',
 'player_other_raw_var_8',
 'player_other_raw_var_9',
 'player_other_raw_var_10',
 'player_offensive_raw_var_11',
 'player_other_raw_var_12',
 'player_other_raw_var_13',
 'player_other_raw_var_14',
 'player_positional_raw_var_15',
 'player_positional_raw_var_16',
 'player_positional_raw_var_17',
 'player_other_raw_var_19',
 'player_other_raw_var_20',
 'player_positional_raw_var_21',
 'player_positional_raw_var_22',
 'player_positional_raw_var_23',
 'player_other_raw_var_24',
 'player_other_raw_var_25',
 'player_other_raw_var_26',
 'player_other_raw_var_27',
 'player_other_raw_var_28',
 'player_other_raw_var_29',
 'player_other_raw_var_30',
 'player_other_raw_var_33',
 'player_other_raw_var_36',
 'player_offensive_raw_var_46',
 'player_ot

In [47]:
X = train.data.drop(["rating_num", "row_id"], axis=1)
y = train.data["rating_num"]

In [48]:
X = X.fillna(0)

In [49]:
for col in X.columns:
    if X[col].dtype == 'object':
        print(col)

winner
team


In [52]:
categorical_features_indices = []

for i, col in enumerate(X.columns):
    if col in categorical_features:
        categorical_features_indices.append(i)
        if X[col].dtype != 'object':
            X[col] = X[col].astype("int")

In [53]:
classifier = CatBoostClassifier()
classifier.fit(X, y, categorical_features_indices)

Learning rate set to 0.092294
0:	learn: 2.5068775	total: 577ms	remaining: 9m 36s
1:	learn: 2.4608946	total: 1.09s	remaining: 9m 5s
2:	learn: 2.4214637	total: 1.55s	remaining: 8m 36s
3:	learn: 2.3880641	total: 1.98s	remaining: 8m 12s
4:	learn: 2.3583731	total: 2.46s	remaining: 8m 9s
5:	learn: 2.3228508	total: 3.01s	remaining: 8m 18s
6:	learn: 2.2970440	total: 3.45s	remaining: 8m 10s
7:	learn: 2.2769898	total: 3.97s	remaining: 8m 12s
8:	learn: 2.2601457	total: 4.41s	remaining: 8m 6s
9:	learn: 2.2432056	total: 4.91s	remaining: 8m 6s
10:	learn: 2.2228916	total: 5.38s	remaining: 8m 3s
11:	learn: 2.2109824	total: 5.87s	remaining: 8m 3s
12:	learn: 2.1992044	total: 6.36s	remaining: 8m 3s
13:	learn: 2.1847383	total: 6.84s	remaining: 8m 1s
14:	learn: 2.1751301	total: 7.37s	remaining: 8m 4s
15:	learn: 2.1553279	total: 7.9s	remaining: 8m 6s
16:	learn: 2.1440544	total: 8.42s	remaining: 8m 7s
17:	learn: 2.1365604	total: 8.88s	remaining: 8m 4s
18:	learn: 2.1291783	total: 9.4s	remaining: 8m 5s
19:	lea

In [57]:
pd.DataFrame({'feature_importance': classifier.get_feature_importance(),
              'feature_names': X.columns}).sort_values(by=['feature_importance'],
                                                           ascending=False)

,feature_importance,feature_names
0,8.367301,scout_id
3,5.096853,competitionId
1,1.447305,winner
6,0.964862,player_height
199,0.956541,player_positional_raw_var_185
...,...,...
703,0.000000,team2_other_raw_var_72
43,0.000000,player_other_raw_var_29
186,0.000000,player_other_raw_var_172
290,0.000000,player_offensive_ratio_var_77


In [67]:
row_id_test = test.data['row_id']
test_data = test.data

In [69]:
test_data = test_data.fillna(0)

In [70]:
for col in test_data.columns:
    if col in categorical_features:
        if test_data[col].dtype != 'object':
            test_data[col] = test_data[col].astype("int")

In [81]:
classifier_predicts = classifier.predict(test_data)

In [74]:
pd.DataFrame(predicts, row_id_test, columns=["rating_num"]).to_csv("../submissions/catboost_init.csv")

In [76]:
regressor = CatBoostRegressor()
regressor.fit(X, y, categorical_features_indices, verbose=1000)

Learning rate set to 0.06596
0:	learn: 1.8423747	total: 69.7ms	remaining: 1m 9s
999:	learn: 1.1028166	total: 39.1s	remaining: 0us


In [92]:
regressor_predicts = regressor.predict(test_data)

In [86]:
predicts = pd.DataFrame(regressor_predicts, row_id_test, columns=["regressor_predicts"])
predicts["classifier_predicts"] = classifier_predicts

In [95]:
predicts.head(20)

,regressor_predicts,classifier_predicts
row_id,,
14785,6.925193,10.0
15125,5.151694,3.0
13095,5.969641,5.0
6079,6.002569,5.0
16584,6.387818,6.5
18508,5.700755,6.0
18731,5.867504,6.0
13610,7.398130,7.5
21768,4.029116,6.5


In [96]:
predicts["combined"] = (predicts["regressor_predicts"] + predicts["classifier_predicts"]) / 2

In [97]:
predicts.head(20)

,regressor_predicts,classifier_predicts,combined
row_id,,,
14785,6.925193,10.0,8.462596
15125,5.151694,3.0,4.075847
13095,5.969641,5.0,5.484821
6079,6.002569,5.0,5.501285
16584,6.387818,6.5,6.443909
18508,5.700755,6.0,5.850377
18731,5.867504,6.0,5.933752
13610,7.398130,7.5,7.449065
21768,4.029116,6.5,5.264558


In [93]:
regressor_predicts_rounded = list(map(lambda x: 10 if x > 10 else round(x * 2) / 2, regressor_predicts))

In [94]:
pd.DataFrame(regressor_predicts_rounded, row_id_test, columns=["rating_num"]).to_csv("../submissions/catboost_init_regression_rounded.csv")
pd.DataFrame(regressor_predicts, row_id_test, columns=["rating_num"]).to_csv("../submissions/catboost_init_regression.csv")

In [100]:
pd.DataFrame(predicts["combined"].values, row_id_test, columns=["rating_num"]).to_csv("../submissions/catboost_init_combined.csv")

In [101]:
grouped = pd.read_csv("../data/helpers/grouped.csv")

In [104]:
separated_groups = grouped.head(8)[["competitionId", "scout_id"]]

In [118]:
filtered_categorical_features = list(filter(lambda x: x not in ["competitionId", "scout_id"], categorical_features))

separate_categorical_feature_indices = []
for i, col in enumerate(X.drop(["competitionId", "scout_id"], axis=1).columns):
    if col in filtered_categorical_features:
        separate_categorical_feature_indices.append(i)

In [121]:
test_data = test.data
test_data = test_data.fillna(0)

for col in test_data.columns:
    if col in filtered_categorical_features:
        if test_data[col].dtype != 'object':
            test_data[col] = test_data[col].astype("int")

In [122]:
prepared_indices = []
predicts_separated = []
for competition_id, scout_id in separated_groups.values:
    regressor_inner = CatBoostRegressor()
    X_inner = X[(X["competitionId"] == competition_id) & (X["scout_id"] == scout_id)]
    X_inner = X_inner.drop(["competitionId", "scout_id"], axis=1)
    y_inner = y[X_inner.index]
    regressor_inner.fit(X_inner, y_inner, separate_categorical_feature_indices, verbose=500)

    test_data_inner = test_data[(test_data["competitionId"] == competition_id) & (test_data["scout_id"] == scout_id)]
    test_data_inner = test_data_inner.drop(["competitionId", "scout_id"], axis=1)
    row_id_test_inner = test_data_inner["row_id"]
    test_data_inner = test_data_inner.drop("row_id", axis=1)
    predicts_inner = regressor_inner.predict(test_data_inner)
    predicts_separated += list(predicts_inner)
    prepared_indices += list(row_id_test_inner.values)

Learning rate set to 0.04265
0:	learn: 2.8958602	total: 22.1ms	remaining: 22.1s
500:	learn: 0.7837229	total: 12.7s	remaining: 12.7s
999:	learn: 0.3213546	total: 24.7s	remaining: 0us
Learning rate set to 0.04076
0:	learn: 2.1381576	total: 20.4ms	remaining: 20.4s
500:	learn: 0.6006471	total: 11s	remaining: 11s
999:	learn: 0.2138524	total: 22s	remaining: 0us
Learning rate set to 0.040706
0:	learn: 1.3465734	total: 19.9ms	remaining: 19.9s
500:	learn: 0.2722709	total: 10.1s	remaining: 10.1s
999:	learn: 0.0830441	total: 20.2s	remaining: 0us
Learning rate set to 0.04066
0:	learn: 1.1346233	total: 21.1ms	remaining: 21.1s
500:	learn: 0.3403405	total: 10.8s	remaining: 10.8s
999:	learn: 0.1150207	total: 21.6s	remaining: 0us
Learning rate set to 0.03971
0:	learn: 1.6651870	total: 18.9ms	remaining: 18.9s
500:	learn: 0.4825198	total: 10s	remaining: 9.98s
999:	learn: 0.1617400	total: 19.9s	remaining: 0us
Learning rate set to 0.03971
0:	learn: 1.5764335	total: 22.7ms	remaining: 22.7s
500:	learn: 0.282

In [148]:
predicts_separated = pd.DataFrame(predicts_separated, prepared_indices, columns=["separated_predicts"])

In [153]:
predicts_separated.shape

(2680, 1)

In [126]:
test_data_remain = test_data[~test_data.index.isin(prepared_indices)]

In [127]:
test_data_remain_row_id = test_data_remain["row_id"]

In [128]:
predicts.head()

,regressor_predicts,classifier_predicts,combined
row_id,,,
14785,6.925193,10.0,8.462596
15125,5.151694,3.0,4.075847
13095,5.969641,5.0,5.484821
6079,6.002569,5.0,5.501285
16584,6.387818,6.5,6.443909


In [150]:
predicts = predicts.join(predicts_separated)
predicts["separated_predicts"] = predicts["separated_predicts"].fillna(predicts["regressor_predicts"])

In [151]:
predicts.head(20)

,regressor_predicts,classifier_predicts,combined,separated_predicts
row_id,,,,
14785,6.925193,10.0,8.462596,7.326513
15125,5.151694,3.0,4.075847,5.151694
13095,5.969641,5.0,5.484821,5.969641
6079,6.002569,5.0,5.501285,5.841775
16584,6.387818,6.5,6.443909,6.115334
18508,5.700755,6.0,5.850377,5.700755
18731,5.867504,6.0,5.933752,6.605344
13610,7.398130,7.5,7.449065,8.067277
21768,4.029116,6.5,5.264558,4.029116


In [152]:
pd.DataFrame(predicts["separated_predicts"].values, row_id_test, columns=["rating_num"]).to_csv("../submissions/catboost_separated.csv")

In [154]:
filtered_categorical_features = list(filter(lambda x: x not in ["scout_id"], categorical_features))

separate_categorical_feature_indices = []
for i, col in enumerate(X.drop(["scout_id"], axis=1).columns):
    if col in filtered_categorical_features:
        separate_categorical_feature_indices.append(i)

In [155]:
scout_id_groups = pd.read_csv("../data/helpers/scout_id_grouped.csv")

In [157]:
scout_id_groups.head(10)

,scout_id,row_id
0,16,3765
1,13,2790
2,3,2740
3,11,2302
4,15,2026
5,1,1973
6,4,1023
7,12,982
8,2,926
9,10,553


In [158]:
prepared_indices = []
predicts_separated = []
for scout_id in scout_id_groups.head(10)["scout_id"].values:
    regressor_inner = CatBoostRegressor()
    X_inner = X[(X["scout_id"] == scout_id)]
    X_inner = X_inner.drop(["scout_id"], axis=1)
    y_inner = y[X_inner.index]
    regressor_inner.fit(X_inner, y_inner, separate_categorical_feature_indices, verbose=500)

    test_data_inner = test_data[(test_data["scout_id"] == scout_id)]
    test_data_inner = test_data_inner.drop(["scout_id"], axis=1)
    row_id_test_inner = test_data_inner["row_id"]
    test_data_inner = test_data_inner.drop("row_id", axis=1)
    predicts_inner = regressor_inner.predict(test_data_inner)
    predicts_separated += list(predicts_inner)
    prepared_indices += list(row_id_test_inner.values)

Learning rate set to 0.050484
0:	learn: 1.8544222	total: 31.5ms	remaining: 31.5s
500:	learn: 0.9655595	total: 15.1s	remaining: 15s
999:	learn: 0.6109411	total: 29.2s	remaining: 0us
Learning rate set to 0.048149
0:	learn: 1.5286631	total: 31.8ms	remaining: 31.8s
500:	learn: 0.6955817	total: 14.8s	remaining: 14.7s
999:	learn: 0.3785931	total: 29.6s	remaining: 0us
Learning rate set to 0.048011
0:	learn: 1.7725561	total: 31.7ms	remaining: 31.7s
500:	learn: 0.8084058	total: 15.1s	remaining: 15s
999:	learn: 0.4384899	total: 29.8s	remaining: 0us
Learning rate set to 0.046708
0:	learn: 1.7169492	total: 33.5ms	remaining: 33.5s
500:	learn: 0.6585590	total: 14.3s	remaining: 14.2s
999:	learn: 0.3489788	total: 28.4s	remaining: 0us
Learning rate set to 0.045775
0:	learn: 1.2785372	total: 33.9ms	remaining: 33.9s
500:	learn: 0.5632894	total: 14.2s	remaining: 14.1s
999:	learn: 0.2796331	total: 28.3s	remaining: 0us
Learning rate set to 0.045584
0:	learn: 2.7135205	total: 30.5ms	remaining: 30.4s
500:	lea

In [159]:
scout_id_predicts = pd.DataFrame(predicts_separated, prepared_indices, columns=["scout_id_predicts"])

In [162]:
predicts = predicts.join(scout_id_predicts)
predicts["scout_id_predicts"] = predicts["scout_id_predicts"].fillna(predicts["regressor_predicts"])

In [163]:
pd.DataFrame(predicts["scout_id_predicts"].values, row_id_test, columns=["rating_num"]).to_csv("../submissions/catboost_scout_id_grouped.csv")